# STORE MODELS BY CITY

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel 
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [3]:
import pyodbc

# Create a connection string
conn_str = (
    r'DRIVER={...};'
    r'SERVER=...;'  
    r'DATABASE=Retail;'  
    r'UID=...;'  
    r'PWD=...'  
)

# Establish a connection
conn = pyodbc.connect(conn_str)

queries = ["SELECT * FROM WEEKLY_CITY WHERE city_id = 'C014' ORDER BY date", "SELECT * FROM DAILY_CITY WHERE city_id = 'C014' ORDER BY date",
           "SELECT * FROM WEEKLY_CITY WHERE city_id = 'C022' ORDER BY date", "SELECT * FROM DAILY_CITY WHERE city_id = 'C022' ORDER BY date", 
           "SELECT * FROM WEEKLY_CITY WHERE city_id = 'C031' ORDER BY date", "SELECT * FROM DAILY_CITY WHERE city_id = 'C031' ORDER BY date", 
           "SELECT * FROM WEEKLY_CITY WHERE city_id = 'C024' ORDER BY date", "SELECT * FROM DAILY_CITY WHERE city_id = 'C024' ORDER BY date",
           "SELECT * FROM View_Weekly_Product_Store AS V JOIN PRODUCT AS P ON V.product_id = P.product_id ORDER BY date"] 

list_df_names = ['df_weekly_c14', 'df_daily_c14', 
                 'df_weekly_c22', 'df_daily_c22',  
                 'df_weekly_c31', 'df_daily_c31', 
                 'df_weekly_c24', 'df_daily_c24',
                 'df_weekly_prod']  

if len(queries) != len(list_df_names):
    raise ValueError('The number of queries does not match the number of dataframe names.')

# Loop over the queries
for query, df_name in zip(queries, list_df_names):
    df = pd.read_sql(query, conn)
    
    globals()[df_name] = df

C:\Users\Furkan\AppData\Local\Temp\ipykernel_6708\2800071476.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [24]:
%run functions/Functions_Modified-Retail.ipynb

# DAILY PREDICTIONS

# 1. DAILY C14 

## 1.1. DAILY C14 SALES PREDICTION

In [132]:
df = df_daily_c14.copy()

In [133]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')
df = create_rolling_features(df, df['total_sales'])

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')
    
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [134]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [28]:
model_daily_c14_sales = pipe_regressor(X_train, y_train, XGBRegressor(random_state=42))
y_pred = model_daily_c14_sales.predict(X_test)
get_metrics(y_test, y_pred)

,RMSE,R2
0,12.741385,0.847271


In [143]:
daily_sales_c14 = features.copy()
daily_sales_c14

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,97,5654.0,11.363146,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,67,1568.0,7.173750,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,99,612.0,5.684800,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,73,1068.0,7.205758,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,22,1374.0,7.053462,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,73,1069.0,19.551250,0,3,4,10,2019,304,31,44,11.26,235.130000,239.83,125.29,431.27,108.594325
2019-10-31,7,817.0,15.156897,0,3,4,10,2019,304,31,44,431.27,186.821429,173.26,93.11,298.44,77.587713
2019-10-31,97,5089.0,31.794286,0,3,4,10,2019,304,31,44,252.51,260.100000,173.26,93.11,765.46,234.178879


In [135]:
trial = features[(features.index == '2019-06-09') & (features['store_id'] == 3)]
trial

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2019-06-09,3,1172.0,17.618971,0,6,2,6,2019,160,9,23,10.0,49.714286,32.0,19.0,100.0,31.982882


In [136]:
model_daily_c14_sales.predict(trial)

array([40.301373], dtype=float32)

In [31]:
model_daily_c14_sales.predict(X_test).min()

4.7027726

In [32]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']
pred_oos = model_daily_c14_sales.predict(X_oos)
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,15.776275,0.896744


## 1.2. DAILY C14 REVENUE PREDICTION

In [139]:
df = df_daily_c14.copy()

In [140]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')
df = create_rolling_features(df, df['total_revenue'])

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [141]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [36]:
model_daily_c14_rev = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))
y_pred = model_daily_c14_rev.predict(X_test)
get_metrics(y_test, y_pred)

Learning rate set to 0.06793
0:	learn: 238.5576244	total: 184ms	remaining: 3m 4s
1:	learn: 226.4211438	total: 198ms	remaining: 1m 38s
2:	learn: 214.6860175	total: 210ms	remaining: 1m 9s
3:	learn: 203.9905205	total: 221ms	remaining: 55s
4:	learn: 194.1302756	total: 233ms	remaining: 46.3s
5:	learn: 185.1442599	total: 246ms	remaining: 40.8s
6:	learn: 176.6590555	total: 257ms	remaining: 36.5s
7:	learn: 168.7250593	total: 265ms	remaining: 32.9s
8:	learn: 161.6358525	total: 273ms	remaining: 30.1s
9:	learn: 155.3585288	total: 280ms	remaining: 27.7s
10:	learn: 149.5199537	total: 286ms	remaining: 25.7s
11:	learn: 144.0806826	total: 295ms	remaining: 24.3s
12:	learn: 139.3914617	total: 301ms	remaining: 22.8s
13:	learn: 134.9528979	total: 308ms	remaining: 21.7s
14:	learn: 131.0813943	total: 313ms	remaining: 20.5s
15:	learn: 127.5637413	total: 317ms	remaining: 19.5s
16:	learn: 124.1923853	total: 322ms	remaining: 18.6s
17:	learn: 121.3425486	total: 327ms	remaining: 17.8s
18:	learn: 118.3075379	total

,RMSE,R2
0,86.423428,0.868345


In [142]:
daily_revenue_c14 = features.copy()
daily_revenue_c14

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,97,5654.0,11.363146,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,67,1568.0,7.173750,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,99,612.0,5.684800,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,73,1068.0,7.205758,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,22,1374.0,7.053462,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,73,1069.0,19.551250,0,3,4,10,2019,304,31,44,11.26,235.130000,239.83,125.29,431.27,108.594325
2019-10-31,7,817.0,15.156897,0,3,4,10,2019,304,31,44,431.27,186.821429,173.26,93.11,298.44,77.587713
2019-10-31,97,5089.0,31.794286,0,3,4,10,2019,304,31,44,252.51,260.100000,173.26,93.11,765.46,234.178879


In [37]:
model_daily_c14_rev.predict(X_test).min()

39.54036634614448

In [38]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']
pred_oos = model_daily_c14_rev.predict(X_oos)
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,65.650013,0.938459


In [39]:
pred_oos.min()

29.08376510883835

# 2. DAILY C22 

In [40]:
%run functions/Functions_Modified-Retail.ipynb

## 2.1. DAILY C22 SALES PREDICTION

In [147]:
df = df_daily_c22.copy()

In [148]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')
df = create_rolling_features(df, df['total_sales'])

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [149]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [44]:
model_daily_c22_sales = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))
y_pred = model_daily_c22_sales.predict(X_test)
get_metrics(y_test, y_pred)

Learning rate set to 0.062845
0:	learn: 19.0007228	total: 7.22ms	remaining: 7.21s
1:	learn: 18.4675940	total: 13.1ms	remaining: 6.55s
2:	learn: 17.9978716	total: 18.9ms	remaining: 6.27s
3:	learn: 17.5950000	total: 26.2ms	remaining: 6.53s
4:	learn: 17.1627738	total: 32.8ms	remaining: 6.53s
5:	learn: 16.8004086	total: 38.6ms	remaining: 6.39s
6:	learn: 16.4263898	total: 44.2ms	remaining: 6.27s
7:	learn: 16.1298368	total: 49.4ms	remaining: 6.13s
8:	learn: 15.8740281	total: 55.7ms	remaining: 6.13s
9:	learn: 15.6220764	total: 60.8ms	remaining: 6.02s
10:	learn: 15.4057865	total: 65.7ms	remaining: 5.91s
11:	learn: 15.1928525	total: 69.9ms	remaining: 5.75s
12:	learn: 15.0052600	total: 74.1ms	remaining: 5.62s
13:	learn: 14.8152169	total: 77.9ms	remaining: 5.48s
14:	learn: 14.6542348	total: 81.3ms	remaining: 5.34s
15:	learn: 14.4454855	total: 84ms	remaining: 5.17s
16:	learn: 14.2983724	total: 87.7ms	remaining: 5.07s
17:	learn: 14.1465364	total: 91ms	remaining: 4.96s
18:	learn: 14.0226107	total: 9

,RMSE,R2
0,7.727134,0.651577


In [152]:
daily_sales_c22 = features.copy()
daily_sales_c22

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,140,1177.0,5.346949,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,34,721.0,6.537368,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,103,1260.0,7.514040,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,144,1052.0,8.023208,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,27,1116.0,7.060748,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,125,1853.0,18.278407,0,3,4,10,2019,304,31,44,11.0,23.714286,28.0,6.0,37.0,11.968212
2019-10-31,103,1222.0,16.618819,0,3,4,10,2019,304,31,44,35.0,22.428571,26.0,6.0,37.0,10.997835
2019-10-31,35,1188.0,14.843426,0,3,4,10,2019,304,31,44,15.0,24.428571,28.0,6.0,37.0,10.690450


In [45]:
model_daily_c22_sales.predict(X_test).min()

9.915249561736786

In [46]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']
pred_oos = model_daily_c22_sales.predict(X_oos)
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,8.326639,0.788334


In [47]:
pred_oos.min()

7.71800978382452

## 2.2. DAILY C22 REVENUE PREDICTION

In [153]:
df = df_daily_c22.copy()

In [154]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')
df = create_rolling_features(df, df['total_revenue'])

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [155]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [51]:
model_daily_c22_rev = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))
y_pred = model_daily_c22_rev.predict(X_test)
get_metrics(y_test, y_pred)

Learning rate set to 0.062845
0:	learn: 78.5452385	total: 7.3ms	remaining: 7.29s
1:	learn: 76.3679735	total: 12.2ms	remaining: 6.09s
2:	learn: 74.4966641	total: 16ms	remaining: 5.33s
3:	learn: 72.7273706	total: 21.4ms	remaining: 5.33s
4:	learn: 71.1252630	total: 26.1ms	remaining: 5.2s
5:	learn: 69.6831304	total: 31ms	remaining: 5.13s
6:	learn: 68.3328546	total: 39.3ms	remaining: 5.57s
7:	learn: 67.0862028	total: 46.2ms	remaining: 5.72s
8:	learn: 65.7245225	total: 55.8ms	remaining: 6.14s
9:	learn: 64.6056980	total: 65.2ms	remaining: 6.46s
10:	learn: 63.5651620	total: 74.2ms	remaining: 6.67s
11:	learn: 62.6722799	total: 82ms	remaining: 6.75s
12:	learn: 61.8703024	total: 90.1ms	remaining: 6.84s
13:	learn: 61.0675942	total: 94.8ms	remaining: 6.68s
14:	learn: 60.4000549	total: 100ms	remaining: 6.58s
15:	learn: 59.7947369	total: 104ms	remaining: 6.38s
16:	learn: 59.1527605	total: 107ms	remaining: 6.19s
17:	learn: 58.5219217	total: 111ms	remaining: 6.04s
18:	learn: 57.9788205	total: 117ms	rem

,RMSE,R2
0,49.193754,0.610012


In [156]:
daily_revenue_c22 = features.copy()
daily_revenue_c22

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,140,1177.0,5.346949,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,34,721.0,6.537368,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,103,1260.0,7.514040,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,144,1052.0,8.023208,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,27,1116.0,7.060748,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,125,1853.0,18.278407,0,3,4,10,2019,304,31,44,60.84,125.020000,153.46,35.18,191.28,64.153617
2019-10-31,103,1222.0,16.618819,0,3,4,10,2019,304,31,44,153.46,116.914286,96.72,35.18,191.28,63.542975
2019-10-31,35,1188.0,14.843426,0,3,4,10,2019,304,31,44,68.25,154.045714,174.66,35.18,328.17,97.327472


In [52]:
model_daily_c22_rev.predict(X_test).min()

41.226853251535516

In [53]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']
pred_oos = model_daily_c22_rev.predict(X_oos)
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,37.725454,0.779239


In [54]:
pred_oos.min()

11.845062603327307

# 3. DAILY C31 

## 3.1. DAILY C31 SALES PREDICTION

In [157]:
df = df_daily_c31.copy()

In [158]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')             ##!!
df = create_rolling_features(df, df['total_sales'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [159]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [58]:
model_daily_c31_sales = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_daily_c31_sales.predict(X_test)                                           ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.058236
0:	learn: 27.3536703	total: 4.38ms	remaining: 4.37s
1:	learn: 26.4331841	total: 8ms	remaining: 3.99s
2:	learn: 25.6011041	total: 11.1ms	remaining: 3.69s
3:	learn: 24.8617457	total: 14.1ms	remaining: 3.52s
4:	learn: 24.1488648	total: 17.9ms	remaining: 3.57s
5:	learn: 23.4710411	total: 21.4ms	remaining: 3.54s
6:	learn: 22.8243772	total: 24.7ms	remaining: 3.51s
7:	learn: 22.2407033	total: 27.9ms	remaining: 3.46s
8:	learn: 21.7059144	total: 30.7ms	remaining: 3.38s
9:	learn: 21.1973992	total: 34.9ms	remaining: 3.45s
10:	learn: 20.7166674	total: 38.1ms	remaining: 3.42s
11:	learn: 20.2859900	total: 41.2ms	remaining: 3.39s
12:	learn: 19.9009958	total: 44.8ms	remaining: 3.4s
13:	learn: 19.5211521	total: 48ms	remaining: 3.38s
14:	learn: 19.1545841	total: 51.5ms	remaining: 3.38s
15:	learn: 18.8486717	total: 54.8ms	remaining: 3.37s
16:	learn: 18.5311278	total: 58.5ms	remaining: 3.38s
17:	learn: 18.2671793	total: 61.5ms	remaining: 3.35s
18:	learn: 18.0093149	total: 64m

,RMSE,R2
0,11.605701,0.730386


In [161]:
daily_sales_c31 = features.copy()
daily_sales_c31

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,49,2285.0,8.509161,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,87,1912.0,7.951884,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,116,708.0,5.428103,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,119,547.0,5.933276,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,11,605.0,6.030727,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,48,8414.0,32.370379,0,3,4,10,2019,304,31,44,26.0,31.571429,29.0,23.0,44.0,8.223080
2019-10-31,53,817.0,13.113455,0,3,4,10,2019,304,31,44,26.0,30.714286,29.0,20.0,44.0,9.159954
2019-10-31,1,4025.0,24.805625,0,3,4,10,2019,304,31,44,23.0,41.142857,31.0,20.0,96.0,25.641299


In [59]:
model_daily_c31_sales.predict(X_test).min()

11.418619250526518

In [60]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']                       ##!!
pred_oos = model_daily_c31_sales.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,10.701417,0.836073


In [61]:
pred_oos.min()

8.892428803162474

## 3.2. DAILY C31 REVENUE PREDICTION

In [162]:
df = df_daily_c31.copy()

In [163]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')             ##!!
df = create_rolling_features(df, df['total_revenue'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [164]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [65]:
model_daily_c31_rev = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_daily_c31_rev.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.058236
0:	learn: 150.9140258	total: 11.8ms	remaining: 11.8s
1:	learn: 145.6783111	total: 16.6ms	remaining: 8.28s
2:	learn: 140.8985821	total: 19.6ms	remaining: 6.5s
3:	learn: 136.3031208	total: 22.8ms	remaining: 5.69s
4:	learn: 132.1346440	total: 26.1ms	remaining: 5.18s
5:	learn: 128.3628789	total: 29.7ms	remaining: 4.91s
6:	learn: 124.6205526	total: 33ms	remaining: 4.67s
7:	learn: 121.1307423	total: 36.3ms	remaining: 4.5s
8:	learn: 117.9904051	total: 41.2ms	remaining: 4.54s
9:	learn: 114.7609898	total: 47.1ms	remaining: 4.66s
10:	learn: 111.8510520	total: 50.4ms	remaining: 4.53s
11:	learn: 109.3390802	total: 54.6ms	remaining: 4.5s
12:	learn: 106.8531843	total: 57.9ms	remaining: 4.39s
13:	learn: 104.8117866	total: 61.6ms	remaining: 4.34s
14:	learn: 102.8346861	total: 65.1ms	remaining: 4.28s
15:	learn: 101.0736643	total: 68.3ms	remaining: 4.2s
16:	learn: 99.3572806	total: 71.2ms	remaining: 4.12s
17:	learn: 97.7511613	total: 74.2ms	remaining: 4.05s
18:	learn: 96.08

,RMSE,R2
0,76.519573,0.797358


In [165]:
daily_revenue_c31 = features.copy()
daily_revenue_c31

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,49,2285.0,8.509161,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,87,1912.0,7.951884,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,116,708.0,5.428103,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,119,547.0,5.933276,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,11,605.0,6.030727,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,48,8414.0,32.370379,0,3,4,10,2019,304,31,44,165.79,207.084286,189.61,87.05,448.67,130.690552
2019-10-31,53,817.0,13.113455,0,3,4,10,2019,304,31,44,87.05,213.661429,189.61,96.83,448.67,124.662367
2019-10-31,1,4025.0,24.805625,0,3,4,10,2019,304,31,44,106.58,294.295714,233.45,96.83,671.02,202.254563


In [66]:
model_daily_c31_rev.predict(X_test).min()

47.775475099832875

In [67]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']                       ##!!
pred_oos = model_daily_c31_rev.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,67.072645,0.788163


# 4. DAILY C24 

## 4.1. DAILY C24 SALES PREDICTION

In [166]:
df = df_daily_c24.copy()

In [167]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')             ##!!
df = create_rolling_features(df, df['total_sales'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [168]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [71]:
model_daily_c24_sales = pipe_regressor(X_train, y_train, LGBMRegressor(random_state=42))  ##!!
y_pred = model_daily_c24_sales.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

,RMSE,R2
0,6.283095,0.882722


In [169]:
daily_sales_c24 = features.copy()
daily_sales_c24

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,21,872.0,5.186349,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,82,1891.0,8.085217,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,25,1667.0,7.135340,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,8,1953.0,9.476887,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,6,684.0,4.135294,0,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,6,363.0,11.711818,0,3,4,10,2019,304,31,44,66.0,20.285714,12.0,5.0,50.0,16.938263
2019-10-31,25,1589.0,15.702636,0,3,4,10,2019,304,31,44,12.0,20.857143,16.0,5.0,50.0,16.677615
2019-10-31,113,2579.0,17.729944,0,3,4,10,2019,304,31,44,20.0,26.428571,16.0,5.0,59.0,22.006493


In [72]:
model_daily_c24_sales.predict(X_test).min()

5.703216944544624

In [73]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']                       ##!!
pred_oos = model_daily_c24_sales.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,9.244603,0.775011


## 4.2. DAILY C24 REVENUE PREDICTION

In [170]:
df = df_daily_c24.copy()

In [171]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')             ##!!
df = create_rolling_features(df, df['total_revenue'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [172]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [173]:
daily_revenue_c24 = features.copy()
daily_revenue_c24

,store_id,total_stock,average_price,isSpecialDay,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,,
2017-01-02,21,872.0,5.186349,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,82,1891.0,8.085217,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,25,1667.0,7.135340,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,8,1953.0,9.476887,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,6,684.0,4.135294,0,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,6,363.0,11.711818,0,3,4,10,2019,304,31,44,359.79,99.185714,54.56,10.76,263.91,92.338118
2019-10-31,25,1589.0,15.702636,0,3,4,10,2019,304,31,44,54.56,97.518571,49.49,10.76,263.91,93.377607
2019-10-31,113,2579.0,17.729944,0,3,4,10,2019,304,31,44,74.90,135.111429,49.49,10.76,338.05,128.949262


In [77]:
model_daily_c24_rev = pipe_regressor(X_train, y_train, LGBMRegressor(random_state=42))  ##!!
y_pred = model_daily_c24_rev.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

,RMSE,R2
0,50.317168,0.784424


In [78]:
model_daily_c24_rev.predict(X_test).min()

24.779335478336442

In [79]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']                       ##!!
pred_oos = model_daily_c24_rev.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,42.561089,0.800194


# WEEKLY PREDICTIONS

# 1. WEEKLY C14

## 1.1. WEEKLY C14 SALES PREDICTION

In [174]:
df = df_weekly_c14.copy()

In [175]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')             ##!!
df = create_rolling_features(df, df['total_sales'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [176]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [177]:
weekly_sales_c14 = features.copy()
weekly_sales_c14

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-23,14,4945.0,5.560825,0,1,1,2017,23,23,4,602.0,392.714286,320.0,167.0,749.0,214.461896
2017-01-23,97,43544.0,11.888149,0,1,1,2017,23,23,4,167.0,558.857143,471.0,176.0,1330.0,389.507565
2017-01-23,143,5893.0,5.632881,0,1,1,2017,23,23,4,176.0,554.857143,471.0,148.0,1330.0,394.209939
2017-01-23,26,31143.0,11.688470,0,1,1,2017,23,23,4,749.0,680.285714,471.0,148.0,1627.0,567.756026
2017-01-23,3,6463.0,6.033730,0,1,1,2017,23,23,4,568.0,643.142857,320.0,148.0,1627.0,584.581454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-14,72,20535.0,19.082787,0,4,10,2019,287,14,42,267.0,412.714286,338.0,75.0,906.0,274.600749
2019-10-14,59,4794.0,13.735198,0,4,10,2019,287,14,42,228.0,403.142857,338.0,75.0,906.0,283.146857
2019-10-14,3,6102.0,14.110160,0,4,10,2019,287,14,42,314.0,395.285714,338.0,75.0,906.0,286.772683


In [83]:
model_weekly_c14_sales = pipe_regressor(X_train, y_train, LGBMRegressor(random_state=42))  ##!!
y_pred = model_weekly_c14_sales.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

,RMSE,R2
0,54.591472,0.922661


In [84]:
model_weekly_c14_sales.predict(X_test).min()

71.6236519073272

In [85]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']                       ##!!
pred_oos = model_weekly_c14_sales.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,76.849016,0.946


## 1.2. WEEKLY C14 REVENUE PREDICTION

In [178]:
df = df_weekly_c14.copy()

In [179]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')             ##!!
df = create_rolling_features(df, df['total_revenue'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [180]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [181]:
weekly_revenue_c14 = features.copy()
weekly_revenue_c14

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-23,14,4945.0,5.560825,0,1,1,2017,23,23,4,4195.44,1790.202857,1128.97,607.43,5066.83,1579.149519
2017-01-23,97,43544.0,11.888149,0,1,1,2017,23,23,4,607.43,2532.438571,2090.18,682.85,5803.08,2074.050528
2017-01-23,143,5893.0,5.632881,0,1,1,2017,23,23,4,682.85,2484.741429,2090.18,348.97,5803.08,2126.842165
2017-01-23,26,31143.0,11.688470,0,1,1,2017,23,23,4,5066.83,2601.257143,2090.18,348.97,5882.44,2306.616872
2017-01-23,3,6463.0,6.033730,0,1,1,2017,23,23,4,1128.97,2533.534286,2090.18,348.97,5882.44,2363.310582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-14,72,20535.0,19.082787,0,4,10,2019,287,14,42,1544.26,2876.912857,2102.27,605.45,7856.81,2378.379547
2019-10-14,59,4794.0,13.735198,0,4,10,2019,287,14,42,1376.63,2821.100000,2102.27,605.45,7856.81,2423.608185
2019-10-14,3,6102.0,14.110160,0,4,10,2019,287,14,42,1890.61,2741.947143,2102.27,605.45,7856.81,2467.708462


In [89]:
model_weekly_c14_rev = pipe_regressor(X_train, y_train, LGBMRegressor(random_state=42))  ##!!
y_pred = model_weekly_c14_rev.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

,RMSE,R2
0,758.361105,0.776237


In [90]:
model_weekly_c14_rev.predict(X_test).min()

468.8253775463043

In [91]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']                       ##!!
pred_oos = model_weekly_c14_rev.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,689.989025,0.849647


# 2. WEEKLY C22

In [92]:
%run functions/Functions_Modified-Retail.ipynb

## 2.1. WEEKLY C22 SALES PREDICTION

In [182]:
df = df_weekly_c22.copy()

In [183]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')             ##!!
df = create_rolling_features(df, df['total_sales'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [184]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [185]:
weekly_sales_c22 = features.copy()
weekly_sales_c22

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-02,35,7173.0,8.334180,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,84,9226.0,7.828130,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,144,6299.0,8.048612,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,103,7531.0,7.543993,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,54,3527.0,5.881534,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-21,98,6575.0,12.528527,0,4,10,2019,294,21,43,196.0,230.428571,245.0,116.0,395.0,92.419077
2019-10-21,121,6136.0,13.298347,0,4,10,2019,294,21,43,245.0,228.000000,228.0,116.0,395.0,92.195445
2019-10-21,93,9797.0,17.959505,0,4,10,2019,294,21,43,116.0,228.714286,228.0,121.0,395.0,91.197066


In [96]:
model_weekly_c22_sales = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_weekly_c22_sales.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.042385
0:	learn: 123.3764746	total: 3.23ms	remaining: 3.22s
1:	learn: 120.8569427	total: 5.65ms	remaining: 2.82s
2:	learn: 118.6252587	total: 8.38ms	remaining: 2.79s
3:	learn: 116.4721369	total: 11ms	remaining: 2.75s
4:	learn: 114.5825832	total: 14.5ms	remaining: 2.89s
5:	learn: 112.6052649	total: 17.5ms	remaining: 2.9s
6:	learn: 110.9307709	total: 20.2ms	remaining: 2.87s
7:	learn: 109.2700749	total: 23ms	remaining: 2.85s
8:	learn: 107.6078474	total: 25.5ms	remaining: 2.81s
9:	learn: 105.4328585	total: 27.8ms	remaining: 2.75s
10:	learn: 104.3190243	total: 29.8ms	remaining: 2.68s
11:	learn: 102.9789548	total: 32ms	remaining: 2.63s
12:	learn: 101.4253255	total: 34.2ms	remaining: 2.6s
13:	learn: 99.9161620	total: 36.2ms	remaining: 2.55s
14:	learn: 98.7787893	total: 38ms	remaining: 2.49s
15:	learn: 97.5686830	total: 40.7ms	remaining: 2.5s
16:	learn: 96.2674441	total: 42.5ms	remaining: 2.46s
17:	learn: 95.1820913	total: 44.6ms	remaining: 2.43s
18:	learn: 94.0767494	to

,RMSE,R2
0,39.796612,0.742517


In [97]:
model_weekly_c22_sales.predict(X_test).min()

108.8212860597352

In [98]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']                       ##!!
pred_oos = model_weekly_c22_sales.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,49.861753,0.834693


## 2.2. WEEKLY C22 REVENUE PREDICTION

In [186]:
df = df_weekly_c22.copy()

In [187]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')             ##!!
df = create_rolling_features(df, df['total_revenue'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [188]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [189]:
weekly_revenue_c22 = features.copy()
weekly_revenue_c22

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-02,35,7173.0,8.334180,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.0,0.000000
2017-01-02,84,9226.0,7.828130,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.0,0.000000
2017-01-02,144,6299.0,8.048612,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.0,0.000000
2017-01-02,103,7531.0,7.543993,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.0,0.000000
2017-01-02,54,3527.0,5.881534,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-21,98,6575.0,12.528527,0,4,10,2019,294,21,43,1068.18,1115.041429,937.09,500.19,2303.3,604.476812
2019-10-21,121,6136.0,13.298347,0,4,10,2019,294,21,43,937.09,1098.458571,839.62,500.19,2303.3,611.721087
2019-10-21,93,9797.0,17.959505,0,4,10,2019,294,21,43,500.19,1129.681429,839.62,681.55,2303.3,580.896631


In [102]:
model_weekly_c22_rev = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_weekly_c22_rev.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.042385
0:	learn: 538.9806515	total: 9.85ms	remaining: 9.84s
1:	learn: 528.2798078	total: 12.5ms	remaining: 6.26s
2:	learn: 517.0275073	total: 14.7ms	remaining: 4.9s
3:	learn: 506.9531665	total: 17.3ms	remaining: 4.31s
4:	learn: 498.0983145	total: 19.9ms	remaining: 3.96s
5:	learn: 489.3351169	total: 22.5ms	remaining: 3.73s
6:	learn: 481.6129557	total: 25.8ms	remaining: 3.66s
7:	learn: 473.9057190	total: 28.5ms	remaining: 3.54s
8:	learn: 465.9756067	total: 31.7ms	remaining: 3.49s
9:	learn: 459.1303830	total: 34.8ms	remaining: 3.45s
10:	learn: 453.2312480	total: 38ms	remaining: 3.41s
11:	learn: 447.0134280	total: 41.2ms	remaining: 3.39s
12:	learn: 441.3700079	total: 44.2ms	remaining: 3.36s
13:	learn: 435.4052903	total: 46.9ms	remaining: 3.3s
14:	learn: 430.2776186	total: 50.6ms	remaining: 3.32s
15:	learn: 424.1875653	total: 53.6ms	remaining: 3.3s
16:	learn: 418.0011478	total: 56.7ms	remaining: 3.28s
17:	learn: 412.5353744	total: 59.8ms	remaining: 3.26s
18:	learn: 40

,RMSE,R2
0,278.742697,0.683188


In [103]:
model_weekly_c22_rev.predict(X_test).min()

385.3467194291327

In [104]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']                       ##!!
pred_oos = model_weekly_c22_rev.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,243.39165,0.806506


# 3. WEEKLY C31

## 3.1. WEEKLY C31 SALES PREDICTION

In [190]:
df = df_weekly_c31.copy()

In [191]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')             ##!!
df = create_rolling_features(df, df['total_sales'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [192]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [193]:
weekly_sales_c31 = features.copy()
weekly_sales_c31

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-02,106,9422.0,7.158234,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,53,4778.0,5.519642,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,117,7332.0,7.626918,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,116,4174.0,5.499684,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,87,11240.0,7.972845,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-28,53,3488.0,13.050079,0,4,10,2019,301,28,44,511.0,240.428571,176.0,98.0,665.0,194.756136
2019-10-28,49,11367.0,18.235782,0,4,10,2019,301,28,44,176.0,242.285714,189.0,98.0,665.0,194.100244
2019-10-28,47,2804.0,12.770493,0,4,10,2019,301,28,44,200.0,228.571429,142.0,98.0,665.0,200.859700


In [108]:
model_weekly_c31_sales = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_weekly_c31_sales.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.039648
0:	learn: 176.3445261	total: 4.9ms	remaining: 4.9s
1:	learn: 172.0349157	total: 8.39ms	remaining: 4.18s
2:	learn: 167.7300998	total: 11.1ms	remaining: 3.69s
3:	learn: 163.7472195	total: 13.4ms	remaining: 3.34s
4:	learn: 159.9584601	total: 15.6ms	remaining: 3.11s
5:	learn: 156.1161261	total: 18ms	remaining: 2.98s
6:	learn: 152.9019961	total: 26.7ms	remaining: 3.79s
7:	learn: 149.5484727	total: 31.1ms	remaining: 3.86s
8:	learn: 146.4751519	total: 42ms	remaining: 4.62s
9:	learn: 143.6501928	total: 47.7ms	remaining: 4.72s
10:	learn: 141.1317561	total: 54.8ms	remaining: 4.93s
11:	learn: 138.2368035	total: 57.4ms	remaining: 4.73s
12:	learn: 135.8051133	total: 59.2ms	remaining: 4.49s
13:	learn: 132.9685816	total: 60.9ms	remaining: 4.29s
14:	learn: 130.7669839	total: 62.6ms	remaining: 4.11s
15:	learn: 128.7661221	total: 64.2ms	remaining: 3.95s
16:	learn: 126.4711410	total: 65.8ms	remaining: 3.81s
17:	learn: 124.6625678	total: 67.3ms	remaining: 3.67s
18:	learn: 122

,RMSE,R2
0,108.587562,0.636581


In [109]:
model_weekly_c31_sales.predict(X_test).min()

134.7482359648238

In [110]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']                       ##!!
pred_oos = model_weekly_c31_sales.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,62.100306,0.891676


## 3.2. WEEKLY C31 REVENUE PREDICTION

In [194]:
df = df_weekly_c31.copy()

In [195]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')             ##!!
df = create_rolling_features(df, df['total_revenue'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [196]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [197]:
weekly_revenue_c31 = features.copy()
weekly_revenue_c31

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-02,106,9422.0,7.158234,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,53,4778.0,5.519642,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,117,7332.0,7.626918,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,116,4174.0,5.499684,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,87,11240.0,7.972845,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-28,53,3488.0,13.050079,0,4,10,2019,301,28,44,4089.21,1470.932857,904.01,496.00,4690.86,1490.557417
2019-10-28,49,11367.0,18.235782,0,4,10,2019,301,28,44,904.01,1497.592857,1090.63,496.00,4690.86,1480.361551
2019-10-28,47,2804.0,12.770493,0,4,10,2019,301,28,44,1190.80,1375.367143,715.70,335.22,4690.86,1543.871587


In [114]:
model_weekly_c31_rev = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_weekly_c31_rev.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.039648
0:	learn: 1015.4254773	total: 6.01ms	remaining: 6.01s
1:	learn: 994.2904445	total: 7.87ms	remaining: 3.93s
2:	learn: 967.5564055	total: 9.87ms	remaining: 3.28s
3:	learn: 941.3759203	total: 12.6ms	remaining: 3.14s
4:	learn: 917.7773977	total: 14.9ms	remaining: 2.96s
5:	learn: 895.2612423	total: 16.9ms	remaining: 2.79s
6:	learn: 875.3269655	total: 18.8ms	remaining: 2.66s
7:	learn: 853.6851697	total: 20.8ms	remaining: 2.57s
8:	learn: 832.3900941	total: 22.4ms	remaining: 2.47s
9:	learn: 814.6984067	total: 24.6ms	remaining: 2.44s
10:	learn: 797.5807342	total: 27.4ms	remaining: 2.46s
11:	learn: 780.2939968	total: 34.7ms	remaining: 2.86s
12:	learn: 764.2214299	total: 36.6ms	remaining: 2.77s
13:	learn: 747.9871130	total: 38.4ms	remaining: 2.7s
14:	learn: 733.7210596	total: 40.1ms	remaining: 2.63s
15:	learn: 718.6462414	total: 42.3ms	remaining: 2.6s
16:	learn: 704.6712532	total: 44.9ms	remaining: 2.59s
17:	learn: 691.4154723	total: 46.7ms	remaining: 2.55s
18:	learn

,RMSE,R2
0,648.040046,0.79259


In [115]:
model_weekly_c31_rev.predict(X_test).min()

476.6176704616788

In [116]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']                       ##!!
pred_oos = model_weekly_c31_rev.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,355.737604,0.896897


# 4. WEEKLY C24

## 4.1. WEEKLY C24 SALES PREDICTION

In [198]:
df = df_weekly_c24.copy()

In [199]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_sales')             ##!!
df = create_rolling_features(df, df['total_sales'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [200]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_sales']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [201]:
weekly_sales_c24 = features.copy()
weekly_sales_c24

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_sales_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-02,82,11183.0,8.128870,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,113,14141.0,9.739957,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,21,5194.0,5.208824,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,25,9738.0,7.144517,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
2017-01-02,8,11483.0,9.511243,0,1,1,2017,2,2,1,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-14,25,10362.0,15.436999,0,4,10,2019,287,14,42,208.0,204.285714,187.0,67.0,308.0,78.555107
2019-10-14,21,6705.0,12.972444,0,4,10,2019,287,14,42,280.0,197.857143,187.0,67.0,308.0,72.969008
2019-10-14,82,13221.0,18.360991,0,4,10,2019,287,14,42,308.0,198.857143,187.0,67.0,315.0,74.756111


In [120]:
model_weekly_c24_sales = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_weekly_c24_sales.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.036026
0:	learn: 126.0675228	total: 4.01ms	remaining: 4.01s
1:	learn: 123.1157438	total: 9.94ms	remaining: 4.96s
2:	learn: 120.4293374	total: 14.4ms	remaining: 4.78s
3:	learn: 117.2983829	total: 18.4ms	remaining: 4.57s
4:	learn: 114.8276484	total: 24.7ms	remaining: 4.92s
5:	learn: 112.1808399	total: 28.9ms	remaining: 4.79s
6:	learn: 109.8674811	total: 31.7ms	remaining: 4.5s
7:	learn: 107.2248234	total: 35.1ms	remaining: 4.36s
8:	learn: 104.9831564	total: 40ms	remaining: 4.41s
9:	learn: 102.6838747	total: 43.6ms	remaining: 4.31s
10:	learn: 100.4958520	total: 46.1ms	remaining: 4.15s
11:	learn: 98.6807562	total: 48.3ms	remaining: 3.97s
12:	learn: 96.7857481	total: 50.7ms	remaining: 3.85s
13:	learn: 95.0150427	total: 53.2ms	remaining: 3.75s
14:	learn: 93.2297434	total: 57.3ms	remaining: 3.76s
15:	learn: 91.2315107	total: 60.4ms	remaining: 3.72s
16:	learn: 89.5256800	total: 64.9ms	remaining: 3.75s
17:	learn: 87.7452450	total: 68.3ms	remaining: 3.72s
18:	learn: 85.8948

,RMSE,R2
0,27.551562,0.898582


In [121]:
model_weekly_c24_sales.predict(X_test).min()

100.23487756617007

In [122]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_sales']                       ##!!
pred_oos = model_weekly_c24_sales.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,47.704123,0.880989


## 4.2. WEEKLY C24 REVENUE PREDICTION

In [202]:
df = df_weekly_c24.copy()

In [203]:
# Data Cleaning and Feature Creation
df['store_id'] = df['store_id'].str.replace('S', '').astype(int)
df = clean_data(df)
df = create_date_features(df)
df = create_lag_features(df, 'total_revenue')             ##!!
df = create_rolling_features(df, df['total_revenue'])     ##!!

for dt in df.dtypes:
    if dt == 'object':
        raise NameError('There is an object datatype in the dataframe')

# Taking a random sample of data  
df_oos = df.sample(n=100, random_state=42)
df = df.drop(df_oos.index)

In [204]:
features = df.drop(['total_sales', 'total_revenue'], axis=1)
target = df['total_revenue']                              ##!!
train_s = round(len(df) * 0.99)
X_train, X_test, y_train, y_test = features[:train_s], features[train_s:], target[:train_s], target[train_s:]

In [205]:
weekly_revenue_c24 = features.copy()
weekly_revenue_c24

,store_id,total_stock,average_price,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,total_revenue_lag_7,rolling_mean_7,rolling_median_7,rolling_min_7,rolling_max_7,rolling_std_7
date,,,,,,,,,,,,,,,,
2017-01-02,82,11183.0,8.128870,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,113,14141.0,9.739957,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,21,5194.0,5.208824,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,25,9738.0,7.144517,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
2017-01-02,8,11483.0,9.511243,0,1,1,2017,2,2,1,0.00,0.000000,0.00,0.00,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-14,25,10362.0,15.436999,0,4,10,2019,287,14,42,1031.98,1123.225714,1132.05,292.61,1844.20,477.018423
2019-10-14,21,6705.0,12.972444,0,4,10,2019,287,14,42,1264.45,1120.500000,1132.05,292.61,1844.20,476.130650
2019-10-14,82,13221.0,18.360991,0,4,10,2019,287,14,42,1844.20,1115.865714,1132.05,292.61,1811.76,468.001199


In [126]:
model_weekly_c24_rev = pipe_regressor(X_train, y_train, CatBoostRegressor(random_state=42))  ##!!
y_pred = model_weekly_c24_rev.predict(X_test)                                                ##!!
get_metrics(y_test, y_pred)

Learning rate set to 0.036026
0:	learn: 583.3885718	total: 1.89ms	remaining: 1.89s
1:	learn: 570.5898226	total: 3.67ms	remaining: 1.83s
2:	learn: 557.6069872	total: 5.23ms	remaining: 1.74s
3:	learn: 545.3766987	total: 6.95ms	remaining: 1.73s
4:	learn: 535.3730195	total: 10.3ms	remaining: 2.06s
5:	learn: 524.2419188	total: 12.6ms	remaining: 2.09s
6:	learn: 514.8652929	total: 14.7ms	remaining: 2.08s
7:	learn: 502.8752231	total: 16.7ms	remaining: 2.07s
8:	learn: 492.2044466	total: 18.4ms	remaining: 2.03s
9:	learn: 482.4926571	total: 20ms	remaining: 1.98s
10:	learn: 473.6202161	total: 21.7ms	remaining: 1.95s
11:	learn: 464.3586297	total: 23.4ms	remaining: 1.92s
12:	learn: 455.3520751	total: 26.3ms	remaining: 1.99s
13:	learn: 448.2365883	total: 27.8ms	remaining: 1.96s
14:	learn: 440.7824365	total: 29.3ms	remaining: 1.93s
15:	learn: 432.7206680	total: 30.8ms	remaining: 1.9s
16:	learn: 424.9904754	total: 32.5ms	remaining: 1.88s
17:	learn: 418.9516232	total: 34.1ms	remaining: 1.86s
18:	learn: 

,RMSE,R2
0,267.445655,0.757133


In [127]:
model_weekly_c24_rev.predict(X_test).min()

451.91552785965905

In [128]:
X_oos = df_oos.drop(['total_sales', 'total_revenue'], axis=1)
y_oos = df_oos['total_revenue']                       ##!!
pred_oos = model_weekly_c24_rev.predict(X_oos)     ##!!
get_metrics(y_oos, pred_oos)

,RMSE,R2
0,226.461973,0.872954


# 5. DUMP MODELS

In [129]:
from joblib import dump, load

In [131]:
dump(model_daily_c14_rev, '.\models\model_daily_c14_rev.joblib')
dump(model_daily_c14_sales, '.\models\model_daily_c14_sales.joblib')
dump(model_daily_c22_rev, '.\models\model_daily_c22_rev.joblib')
dump(model_daily_c22_sales, '.\models\model_daily_c22_sales.joblib')
dump(model_daily_c24_rev, '.\models\model_daily_c24_rev.joblib')
dump(model_daily_c24_sales, '.\models\model_daily_c24_sales.joblib')
dump(model_daily_c31_rev, '.\models\model_daily_c31_rev.joblib')
dump(model_daily_c31_sales, '.\models\model_daily_c31_sales.joblib')
dump(model_weekly_c14_rev, '.\models\model_weekly_c14_rev.joblib')
dump(model_weekly_c14_sales, '.\models\model_weekly_c14_sales.joblib')
dump(model_weekly_c22_rev, '.\models\model_weekly_c22_rev.joblib')
dump(model_weekly_c22_sales, '.\models\model_weekly_c22_sales.joblib')
dump(model_weekly_c24_rev, '.\models\model_weekly_c24_rev.joblib')
dump(model_weekly_c24_sales, '.\models\model_weekly_c24_sales.joblib')
dump(model_weekly_c31_rev, '.\models\model_weekly_c31_rev.joblib')
dump(model_weekly_c31_sales, '.\models\model_weekly_c31_sales.joblib')

['.\\models_joblib\\model_weekly_c31_sales.joblib']

In [ ]:
df_daily_sales.to_csv('.\CSV_files\db_csvs\daily_sales.csv')
df_daily_revenue.to_csv('.\CSV_files\db_csvs\daily_revenue.csv')
df_weekly_sales.to_csv('.\CSV_files\db_csvs\weekly_sales.csv')
df_weekly_revenue.to_csv('.\CSV_files\db_csvs\weekly_revenue.csv')